In [5]:
from create_table_fpdf2 import PDF
from create_table_fpdf2 import report_data

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from glob import glob
from pylab import rcParams
import os
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm 
from datetime import datetime
rcParams['figure.figsize'] = 12,4
rcParams['savefig.dpi'] = 300

now = datetime.now().strftime('%Y%m%d')

In [6]:
day = '20240106'

In [7]:
file_list = [i.replace('\\','/') for i in sorted(glob(f'players/{day}/**/**/report/*csv'))]
file_name = [f"{os.path.basename(i).split('_')[0]}_{os.path.basename(i).split('_')[1]}" for i in file_list]

In [8]:
file_name

['Kimjieon_008', 'Kwonhyukchan_009', 'Jeonseongwoo_006', 'Leejinyong_011']

In [33]:
class PDF(PDF):
    def header(self):
        # Logo
        self.set_font('helvetica', 'I', 8)
        self.cell(0, -10, 'Kookmin University Hitting Report', ln = True)

    # Page footer
    def footer(self):
        # Set position of the footer
        self.set_y(-15)
        # set font
        self.set_font('helvetica', 'I', 8)
        # Page number
        self.cell(0, 10, f"Page {self.page_no()}" + "/" +f"{{nb}}", align = "C")
        # Logo
        self.image(f'imagefile/logo.png',
                160, 280, 50)
    
    # Adding chapter title to start of each chapter
    def chapter_title(self, ch_num, ch_title):
        # set font
        self.set_font('helvetica', '', 12)
        # background color
        self.set_fill_color(200, 220, 255)
        # Chapter title
        chapter_title = f'Chapter {ch_num} : {ch_title}'
        self.cell(0, 5, chapter_title, ln=1, fill=1)
        # line break
        self.ln()

    # Chapter content
    def chapter_body(self):
        # end each chapter
        self.set_font('helvetica', 'I', 12)
        self.cell(0, 5, 'END OF CHAPTER')

    def print_chapter(self, ch_num, ch_title):
        self.add_page()
        self.chapter_title(ch_num, ch_title)
        self.chapter_body()
        
ks_cols = {
            'pelvic_angular_velo' : ['Pelvic', 'r'],
            'torso_angular_velo' : ['Torso', 'g'],
            'upperarm_angular_velo' : ['Upper Arm','b'],
            'forearm_angular_velo' : ['Forearm','y']
}

pel_cols = {
    'pelvic_forward_tilt' : ['Forward Tilt', 'b'],
    'pelvic_lateral_tilt' : ['Lateral Tilt', 'g'],
    'pelvic_rotation'     : ['Rotation', 'r']
}

tor_cols = {
    'torso_forward_tilt' : ['Forward Tilt', 'b'],
    'torso_lateral_tilt' : ['Lateral Tilt', 'g'],
    'torso_rotation'     : ['Rotation', 'r']
}

grf_y_cols = {
    'rear_force_y' : ['Rear Leg', 'b'],
    'lead_force_y' : ['Lead Leg', 'r']
}

ang_one_cols = {
    'x_factor' : 'X FACTOR',
    'shoulder_rotation' : 'SHOULDER ROTATION',
    'shoulder_horizontal_adduction' : 'SHOULDER HORIZONTAL ADDUCTION',
    'elbow_flexion': 'ELBOW FLEXION'
    }

for_one_cols = {
    'lead_force_z' : 'LEAD LEG GROUND REACTION FORCE (Vertical)',
    'rear_force_z' : 'REAR LEG GROUND REACTION FORCE (Vertical)', 
    'lead_moment_z': 'LEAD LEG TORQUE',
    'rear_moment_z': 'REAR LEG TORQUE'
}

for file, name in tqdm(zip(file_list, file_name)):

    if 'right' in file.split('/'):
        hit_type = 'Right'
    elif 'left' in file.split('/'):
        hit_type = 'Left'
        
    df = pd.read_csv(file)
    store = os.path.split(file)[0]
    time = df['time']

    to_time = df['to_time'][0]
    fc_time = df['fc_10_time'][0]
    max_grf_time = df['lead_max_grf_time'][0]
    bc_time = df['bc_time'][0]
    peak_hand_time = df['peak_hand_time'][0]
    
    ball_vel = df['ball_vel'][0]
    ball_dis = df['ball_dis'][0]
    launch_ang = df['launch_ang'][0]

    ks = report_data.kinematic_sequence(df, ks_cols, time, to_time, fc_time, max_grf_time, bc_time, peak_hand_time)
    pel = report_data.pelvic_angle(df, pel_cols, time, to_time, fc_time, max_grf_time, bc_time)
    tor = report_data.torso_angle(df, tor_cols, time, to_time, fc_time, max_grf_time, bc_time)
    ang, xf = report_data.one_angle(df, ang_one_cols, time, to_time, fc_time, max_grf_time, bc_time)
    y = report_data.grf_y(df, grf_y_cols, time, to_time, fc_time, max_grf_time, bc_time)
    rear, lead_tq, lead_z = report_data.one_force(df, for_one_cols, time, to_time, fc_time, max_grf_time, bc_time)

    pdf = PDF("P", "mm","A4")
    pdf.set_auto_page_break(auto = True, margin = 15)    
    # main page
    pdf.add_page()
    pdf.set_font('helvetica', 'BU', 20)
    title = 'Kookmin University Hitting Report'
    pdf.ln(5)
    title_w = pdf.get_string_width(title) + 5
    doc_w = pdf.w
    pdf.set_x((doc_w - title_w)/2)
    pdf.cell(135, 20, title, align = 'C', ln = True)

    # ===========================================================================================================
    # chapter
    ch1_link = pdf.add_link()
    ch2_link = pdf.add_link()
    ch3_link = pdf.add_link()
    ch4_link = pdf.add_link()
    ch5_link = pdf.add_link()
    ch6_link = pdf.add_link()
    ch7_link = pdf.add_link()
    ch8_link = pdf.add_link()
    ch9_link = pdf.add_link()
    ch10_link = pdf.add_link()
    ch11_link = pdf.add_link()
    ch12_link = pdf.add_link()
    ch13_link = pdf.add_link()
    pdf.set_font('helvetica', 'B', 8)
    pdf.cell(0, 2,"",ln=1,align = '')
    pdf.cell(0, 0, f"Name : {name.split('_')[0]}    Date : {now}   Hitter Type : {hit_type}   Ball Velocity : {ball_vel} km/h   Ball Distance : {ball_dis} m   Launch Angle : {launch_ang} Deg", align = 'C', ln = True)
    # pdf.cell(0, 4,"",ln=1,align = '')
    # pdf.cell(0, 0, f"Ball Velocity - {ball_vel} km/h         Ball Distance - {ball_dis} m      Launch Angle - {launch_ang} Deg", align = 'C', ln = True)
    pdf.cell(0, 3,"",ln=1,align = '')
    pdf.image(f'imagefile/analysis.png',x= 30,w=150,h=75)
    pdf.set_font('helvetica', 'B', 17)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.set_fill_color(r=150,g=0,b=0)
    pdf.set_text_color(255,255,255)
    pdf.cell(0, 7, 'KINEMATIC PARAMETERS', fill=True,ln = 1, align = 'C',link = ch1_link)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    pdf.cell(0, 7, '1. KINEMATIC SEQUENCE', ln = 1, align = '', link = ch1_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '2. X FACTOR', ln = 1, align = '', link = ch2_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '3. PELVIC ANGLE', ln = 1, align = '', link = ch3_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '4. TORSO ANGLE', ln = 1, align = '', link = ch4_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '5. SHOULDER ROTATION', ln = 1, align = '', link = ch5_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '6. SHOULDER ABDUCTION', ln = 1, align = '', link = ch6_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '7. ELBOW FLEXION', ln = 1, align = '', link = ch7_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.set_font('helvetica', 'B', 17)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.set_text_color(255,255,255)
    pdf.cell(0, 7, 'KINETIC PARAMETERS',fill=True, ln = 1, align = 'C',link = ch8_link)
    pdf.set_font('helvetica', 'B', 13)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.set_text_color(0,0,0)
    pdf.cell(0, 7, '1. REAR LEG GROUND REACTION FORCE (Vertical)', ln = 1, align = '', link = ch8_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '2. REAR LEG TORQUE', ln = 1, align = '', link = ch9_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '3. REAR & LEAD LEG AP AXIS GROUND REACTION FORCE', ln = 1, align = '', link = ch10_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '4. LEAD LEG GROUND REACTION FORCE (Vertical)', ln = 1, align = '', link = ch11_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.cell(0, 7, '5. LEAD LEG TORQUE', ln = 1, align = '', link = ch12_link)
    pdf.cell(0,2,"",ln=1,align = '')
    pdf.set_font('helvetica', 'B', 17)
    pdf.cell(0,5,"",ln=1,align = '')
    pdf.set_text_color(255,255,255)
    pdf.cell(0, 7, 'FEEDBACK', ln = 1, fill=True, align = 'C',link = ch13_link)
    pdf.set_text_color(0,0,0)
    # ===========================================================================================================
    # kinematic sequence
    peak_pel = ks['peak']['pelvic_angular_velo']; time_pel = ks['time']['pelvic_angular_velo']
    peak_tor = ks['peak']['torso_angular_velo'];time_tor = ks['time']['pelvic_angular_velo']
    peak_upper = ks['peak']['upperarm_angular_velo'];time_upper = ks['time']['upperarm_angular_velo']
    peak_fore = ks['peak']['forearm_angular_velo'];time_fore = ks['time']['forearm_angular_velo']
    total_time = bc_time - fc_time
    pel_time = round(100 * (time_pel - fc_time) / total_time, 2)
    tor_time = round(100 * (time_tor - fc_time) / total_time, 2)
    upper_time = round(100 * (time_upper - fc_time) / total_time, 2)
    fore_time = round(100 * (time_fore - fc_time) / total_time, 2)
    tor_gain = round(peak_tor / peak_pel,2)
    upper_gain = round(peak_upper / peak_tor,2)
    fore_gain = round(peak_fore / peak_upper,2)

    pdf.add_page()
    pdf.set_link(ch1_link, page = 2)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'KINEMATIC SEQUENCE', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica', 'I', 10)
    pdf.cell(0, 0, 'The order and velocity of the pelvis, trunk and arm rotation')
    pdf.image(f'figure/kinematic.png',x=75,y=28,w=133,h=60)
    pdf.image(f'imagefile/kinematic.png',x=17,y=28,w=50,h=60)

    data_as_dict = {"Segment"                 : ["Pelvic",      "Torso",       "Upper Arm",  "Forearm"],
                    "Minor League"            : ["638 - 780", "843 - 988","1102 - 1327","1711 - 2051"],
                    "Peak Velocity"           : [peak_pel, peak_tor,peak_upper,peak_fore],
                    "Timing [BC - FC (100%)]" : [f"{pel_time} %", f"{tor_time} %",f"{upper_time} %",f"{fore_time} %"],
                    "Speed Gain"              : [''      , tor_gain,upper_gain,fore_gain]}
    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Kinematic Sequence',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [30,40,35,55,40],
                    x_start='C')

    # ===========================================================================================================
    # X-Factor
    pdf.set_link(ch2_link, page = 2)
    pdf.set_y(150)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'X FACTOR', ln = True)
    pdf.set_y(156)
    pdf.set_font('helvetica', 'I', 10)
    pdf.cell(0, 0, 'The angle between the trunk and the pelvis with respect to the ground')
    pdf.image(f'figure/x_factor.png',x=75,y=160,w=133,h=50)
    pdf.image(f'imagefile/xfactor.png',x=17,y=160,w=50,h=60)

    data_as_dict = {
                    "at Lead Leg Toe Off" : [ang['to_time']['x_factor']],
                    "at Foot Contact" : [ang['fc_time']['x_factor']],
                    "at Max GRF"      : [ang['max_grf_time']['x_factor']],
                    "at Ball Contact" : [ang['bc_time']['x_factor']]
                    }
    pdf.set_y(221)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'X Factor',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')

    data_as_dict = {
                    "Max"        : [xf['peak']],
                    "Max Timing [BC - FC (100%)]" : [f"{round(100 * (xf['time'] - fc_time) / total_time,2)} %"],
                    }
    pdf.set_y(244)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [100,100],
                    x_start='C')

    # ===========================================================================================================
    # Pelvic Angle
    pdf.add_page()
    pdf.set_link(ch3_link, page = 3)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'PELVIC ANGLE', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The angle of pelvic relative to the ground')
    pdf.image('figure/pelvic.png',x=75,y=35,w=133,h=50)
    pdf.image('imagefile/pelvic_angle.png',x=8,y=35,w=65,h=50)

    data_as_dict = {"Segment"             : ['Forward Tilt','Lateral Tilt','Rotation'],
                    "at Lead Leg Toe Off" : [pel['to_time']['pelvic_forward_tilt'],pel['to_time']['pelvic_lateral_tilt'],pel['to_time']['pelvic_rotation']],
                    "at Foot Contact"     : [pel['fc_time']['pelvic_forward_tilt'],pel['fc_time']['pelvic_lateral_tilt'],pel['fc_time']['pelvic_rotation']],
                    "at Max GRF"          : [pel['max_grf_time']['pelvic_forward_tilt'],pel['max_grf_time']['pelvic_lateral_tilt'],pel['max_grf_time']['pelvic_rotation']],
                    "at Ball Contact"     : [pel['bc_time']['pelvic_forward_tilt'],pel['bc_time']['pelvic_lateral_tilt'],pel['bc_time']['pelvic_rotation']]}
    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Pelvic Angle',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    # ===========================================================================================================
    # Torso Angle
    pdf.set_link(ch4_link, page = 3)
    pdf.set_y(150)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'TORSO ANGLE', ln = True)
    pdf.set_y(156)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The angle of torso relative to the ground')
    pdf.image(f'figure/torso.png',x=75,y=165,w=133,h=50)
    pdf.image(f'imagefile/torso_angle.png',x=8,y=165,w=65,h=50)

    data_as_dict = {"Segment"             : ['Forward Tilt','Lateral Tilt','Rotation'],
                    "at Lead Leg Toe Off" : [tor['to_time']['torso_forward_tilt'],tor['to_time']['torso_lateral_tilt'],tor['to_time']['torso_rotation']],
                    "at Foot Contact"     : [tor['fc_time']['torso_forward_tilt'],tor['fc_time']['torso_lateral_tilt'],tor['fc_time']['torso_rotation']],
                    "at Max GRF"          : [tor['max_grf_time']['torso_forward_tilt'],tor['max_grf_time']['torso_lateral_tilt'],tor['max_grf_time']['torso_rotation']],
                    "at Ball Contact"     : [tor['bc_time']['torso_forward_tilt'],tor['bc_time']['torso_lateral_tilt'],tor['bc_time']['torso_rotation']]}
    pdf.set_y(221)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Torso Angle',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    # ===========================================================================================================
    # LEAD SHOULDER ROTATION
    pdf.add_page()
    pdf.set_link(ch5_link, page = 4)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD SHOULDER ROTATION', ln = True)
    pdf.set_y(19)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'Internal and external rotation of the lead shoulder')
    pdf.image(f'figure/shoulder_rotation.png',x=75,y=20.5,w=133,h=50)
    pdf.image(f'imagefile/shoulder_rotation.png',x=17,y=20.5,w=50,h=50)

    data_as_dict = {
                    "at Lead Leg Toe Off" : [ang['to_time']['shoulder_rotation']],
                    "at Foot Contact" : [ang['fc_time']['shoulder_rotation']],
                    "at Max GRF"      : [ang['max_grf_time']['shoulder_rotation']],
                    "at Ball Contact" : [ang['bc_time']['shoulder_rotation']]
                    }
    pdf.set_y(72)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Shoulder Rotation',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')
    # ===========================================================================================================
    # LEAD SHOULDER Horiziontal ABDUCTION
    pdf.set_link(ch6_link, page = 4)
    pdf.set_y(112)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD SHOULDER HORIZONTAL AD/ABDUCTION', ln = True)
    pdf.set_y(116)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'Vertical plane rotation of the arm relative to the trunk')
    pdf.image(f'figure/shoulder_horizontal_adduction.png',x=75,y=118,w=133,h=50)
    pdf.image(f'imagefile/shoulder_add.png',x=17,y=118,w=50,h=50)

    data_as_dict = {
                    "at Lead Leg Toe Off" : [ang['to_time']['shoulder_horizontal_adduction']],
                    "at Foot Contact" : [ang['fc_time']['shoulder_horizontal_adduction']],
                    "at Max GRF"      : [ang['max_grf_time']['shoulder_horizontal_adduction']],
                    "at Ball Contact" : [ang['bc_time']['shoulder_horizontal_adduction']]
                    }
    pdf.set_y(165)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'X Factor',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')

    # ===========================================================================================================
    # LEAD ELBOW FLEXION
    pdf.set_link(ch7_link, page = 4)
    pdf.set_y(195)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD ELBOW FLEXION', ln = True)
    pdf.set_y(199)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The sagittal plane rotation of the forearm relative to the upper arm')
    pdf.image(f'figure/elbow_flexion.png',x=75,y=201,w=133,h=50)
    pdf.image(f'imagefile/elbow_flexion.png',x=17,y=201,w=50,h=50)

    data_as_dict = {
                    "at Lead Leg Toe Off" : [ang['to_time']['elbow_flexion']],
                    "at Foot Contact" : [ang['fc_time']['elbow_flexion']],
                    "at Max GRF"      : [ang['max_grf_time']['elbow_flexion']],
                    "at Ball Contact" : [ang['bc_time']['elbow_flexion']]
                    }
    pdf.set_y(252)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Leg Knee Flexion/Extension',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')

    # ===========================================================================================================
    # Rear Leg GRF Z
    pdf.add_page()
    pdf.set_link(ch8_link, page = 5)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'REAR LEG GROUND REACTION FORCE (Vertical)', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The ground reaction force vertical to the ground')
    pdf.image(f'figure/rear_force_z.png',x=75,y=28,w=133,h=50)
    pdf.image(f'imagefile/rear_leg_z.png',x=17,y=28,w=50,h=50)

    data_as_dict = {
                    "Max at Loading Phase" : [rear['max_at_loading']['rear_force_z']],
                    "at Lead Leg Toe Off"  : [rear['to_time']['rear_force_z']],
                    "at Foot Contact"     : [rear['fc_time']['rear_force_z']],
                    "at Max GRF"          : [rear['max_grf_time']['rear_force_z']],
                    "at Ball Contact"     : [rear['bc_time']['rear_force_z']]
                    }
    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Ground Reaction Force (Vertical) [Proficient = above 98 %BW ]',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    # ===========================================================================================================
    # Rear Leg Torque
    pdf.set_link(ch9_link, page = 5)
    pdf.set_y(150)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'REAR LEG TORQUE', ln = True)
    pdf.set_y(156)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The rear leg torque relative to the ground')
    pdf.image(f'figure/rear_moment_z.png',x=75,y=160,w=133,h=50)
    pdf.image(f'imagefile/rear_leg_torque.png',x=17,y=160,w=50,h=50)

    data_as_dict = {
                    "Max at Loading Phase" : [rear['max_at_loading']['rear_moment_z']],
                    "at Lead Leg Toe Off"  : [rear['to_time']['rear_moment_z']],
                    "at Foot Contact"     : [rear['fc_time']['rear_moment_z']],
                    "at Max GRF"          : [rear['max_grf_time']['rear_moment_z']],
                    "at Ball Contact"     : [rear['bc_time']['rear_moment_z']]
                    }
    pdf.set_y(221)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Rear Leg Torque [Proficient = 45 Nm]',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [40,40,40,40,40],
                    x_start='C')

    # ===========================================================================================================
    # AP axis GRF Y
    pdf.add_page()
    pdf.set_link(ch10_link, page = 6)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'REAR & LEAD LEG GROUND REACTION FORCE (AP axis)', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The ground reaction force toward to pitcher')
    pdf.image(f'figure/grf_ap.png',x=75,y=28,w=133,h=50)
    pdf.image(f'imagefile/rear_lead_y.png',x=17,y=28,w=50,h=50)

    data_as_dict = {
                "Segment"             : ['Rear Leg','Lead Leg'],
                "Max"                 : [y['max']['rear_force_y'],y['max']['lead_force_y']],
                "at Lead Leg Max"     : [y['lead_max']['rear_force_y'],y['lead_max']['lead_force_y']],
                "Good"                : ['', 'above 70%BW']
                    }
    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Read & Lead Leg AP aixs Ground Reaction Force',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')

    data_as_dict = {
                "X-Axis Timing"       : [f"{y['x_axis_time']} %"],
                "Proficient"          : ['above 70%BW']
                    }
    pdf.set_y(120)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = '',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [100,100],
                    x_start='C')

    # ===========================================================================================================
    # Lead Leg GRF Z
    pdf.set_link(ch11_link, page = 6)
    pdf.set_y(150)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG GROUND REACTION FORCE (Vertical)', ln = True)
    pdf.set_y(156)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The ground reaction force vertical to ground')
    pdf.image(f'figure/lead_force_z.png',x=75,y=160,w=133,h=50)
    pdf.image(f'imagefile/lead_leg_z.png',x=17,y=160,w=50,h=50)

    data_as_dict = {
                    "at Foot Contact"     : [lead_z['fc_time']],
                    "Max"                 : [lead_z['max']],
                    "FC - MAX Time"       : [f"{lead_z['fc-max_time']} s"],
                    "Loading rate"        : [lead_z['loading_rate']]
                    }
    pdf.set_y(221)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Leg Ground Reaction Force (Vertical)',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [50,50,50,50],
                    x_start='C')

    # ===========================================================================================================
    # Lead Leg Torque
    pdf.add_page()
    pdf.set_link(ch12_link, page = 7)
    pdf.set_y(15)
    pdf.set_font('helvetica', 'B', 12)
    pdf.cell(0, 0, 'LEAD LEG TORQUE', ln = True)
    pdf.set_y(21)
    pdf.set_font('helvetica','i',10)
    pdf.cell(0,0,'The lead leg torque relative to the ground')
    pdf.image(f'figure/lead_moment_z.png',x=75,y=28,w=133,h=50)
    pdf.image(f'imagefile/lead_leg_torque.png',x=17,y=28,w=50,h=50)

    data_as_dict = {
                    "Max"             :  [lead_tq['max']],
                    "Max Timing [BC - FC (100%)]"      :  [f"{round(100 * (lead_tq['time'] - fc_time) / total_time,2)} %"],
                    "Good"            :  ['above 100 Nm']
                    }
    pdf.set_y(90)
    title_size = 8
    pdf.set_font(family='Arial', style='B', size=title_size)
    pdf.create_table(table_data = data_as_dict,
                    title = 'Lead Leg Torque',
                    title_size=10,
                    align_header = 'C',
                    align_data = 'C',
                    cell_width = [63,63,63],
                    x_start='C')
    # =============================================================================
    # comment
    pdf.add_font(family = 'kor', style = 'B', fname =  'NanumGothicCoding-Bold.ttf', uni=True)
    pdf.add_font(family = 'kor', style = '', fname =  'NanumGothicCoding.ttf', uni=True)

    c_path, _ = os.path.split(file)
    A, B, C, D = c_path.split('/')[:-1]
    comment_path = os.path.join(A,B,C,D, 'report')
    pdf.set_link(ch13_link, page = 7)
    with open(f'{comment_path}/comments.txt', 'rb') as fh:
        comment_txt = fh.read().decode('utf-8')

    pdf.set_y(125)
    pdf.set_font('Arial', 'B', 15)
    pdf.cell(0, 0, 'FEEDBACK', ln = True, align = 'C')
    
    pdf.set_y(135)
    pdf.set_font('kor', '', 10)
    pdf.multi_cell(0, 8, comment_txt, ln = True, border='True')

    figure_file = glob('figure/*png')
    for figure in figure_file:
        os.remove(figure)

    # ===========================================================================================================
    pdf.output(f'{store}/{day}_{name}_baseball_report.pdf')

0it [00:00, ?it/s]

4it [00:32,  8.19s/it]
